### Revised Simplex Algorithm for Maximization Problem.

In [22]:
import numpy as np
import sys

In [23]:
# general example
# A transpose
A = [[1,2,1], #A_0 or P_1
     [1,3,5], #A_1 or P_2
     [1,0,0], #A_2 or P_3
     [0,1,0], #A_3 or P_4
     [0,0,1]] #A_4 or P_5
m=3
c = [6,8,0,0,0]
b = [10,25,35]  #m=3 because of 3 equality constraints
I = [2,3,4] #index of A matrix which forms the basis matrix i.e index of basic variable
I_ = [0,1]  #index of non-basic variable

x = [0,0,10,25,35]  #primal answer
y = [0,0,0]  #dual answer
obj = 0 #objective function value
E=[[1,0,0,0]] #Eo each element of E will be a eta mat with m+1 element. last elem of eta stores the position of non-identity col.
exit_index = 0
enter_index = 0
e_index = 0
pbar = [0,0,0] #m=3

In [239]:
# unbounded example
# A transpose
A = [[2,4], #A_0 or P_1
     [-2,0], #A_1 or P_2
     [1,0], #A_2 or P_3
     [0,1]] #A_3 or P_4
m=2
c = [4,6,0,0]
b = [6,16]  #m=3 because of 3 equality constraints
I = [2,3] #index of A matrix which forms the basis matrix i.e index of basic variable
I_ = [0,1]  #index of non-basic variable
x = [0,0,6,16]  #primal answer
y = [0,0]  #dual answer
obj = 0 #objective function value
E=[[1,0,0]] #Eo each element of E will be a eta mat with m+1 element. last elem of eta stores the position of non-identity col.
exit_index = 0
enter_index = 0
e_index = 0
pbar = [0,0] #m=3

In [24]:
def cal_coeff(index): #index of non-basic variable
    return c[index] - np.dot(y,np.transpose(A[index]))

# reverse subsitution (backward direction)
def p_solve(e,p): 
    pt = [0]*len(p)
    pt[e[-1]] = p[e[-1]]/e[e[-1]]
    for i in [x for x in range(len(p)) if x != e[-1]]:
        pt[i] = p[i] - e[i]*pt[e[-1]]
    return pt

def cal_pbar(p):
    pt = p
    for ei in E:
        pt = p_solve(ei,pt)
    return pt

def y_solve(e,cb):
    yt = cb
    s = 0
    for i in [x for x in range(len(cb)) if x != e[-1]]:
        s = s + yt[i]*e[i]
    yt[e[-1]] = (cb[e[-1]] - s)/e[e[-1]]
    return yt

def cal_y(cb):
    yt = cb
    for e in reversed(E):
        yt = y_solve(e,yt)
    return yt

def cal_obj(x):
    return np.dot(x,c)

def cal_dual_obj(y):
    return np.dot(y,b)

In [25]:
while(True):
    maximum = -1 #anti cycling using maximum coefficient rule and Bland Rule.
    for i in I_:
        coeff = cal_coeff(i)
        if((coeff>maximum or (coeff == maximum and i<enter_index))and coeff>0): #enforce Bland's rule for anticycling in degenerate cases
            maximum = coeff
            enter_index = i
    if(maximum == -1):
        print("optimal value of objective achieved")
        break #stop the simplex, objective achieved

    pbar = cal_pbar(A[enter_index])
    #if all p are -ve then stop. problem is unbounded check it in the video.
    if all(i <= 0 for i in pbar):
        print("problem is unbounded in direction of :",enter_index+1,"p:",pbar)
        break

    theta = sys.maxsize #have to be positive
    for i in range(m):
        th = x[I[i]]/pbar[i]
        if(((th < theta) or (th==theta and i<exit_index)) and th >0):  #blands's rule
            theta = th
            exit_index = I[i]
            e_index = i
    x[enter_index] = theta
    for i in range(m):
        x[I[i]] = x[I[i]] - pbar[i]*theta
    x[exit_index]  = 0.0
        
    e = [0]*(len(pbar)+1)
    e[-1] = e_index
    for i in range(len(pbar)):
        e[i] = pbar[i]
    E.append(e)

    exitIi = I.index(exit_index)
    I[exitIi] = enter_index

    entryI_i = I_.index(enter_index)
    I_[entryI_i] = exit_index

    obj = cal_obj(x)

    #find the value of dual y
    cb = [c[i] for i in I]
    y = cal_y(cb)
    print(x)
    print(y)
    print()
    #print("primal obj value: ",obj," dual obj value", cal_dual_obj(y))

[0, 7.0, 3.0, 4.0, 0.0]
[0.0, 0, 1.6]

[2.857142857142857, 6.428571428571429, 0.714285714285714, 0.0, 0.0]
[0.0, 3.1428571428571432, -0.28571428571428614]

[4.999999999999998, 5.000000000000002, 0.0, 0.0, 4.999999999999994]
[2.000000000000001, 1.9999999999999993, 1.7763568394002506e-16]

optimal value of objective achieved


In [6]:
x

[4.999999999999998, 5.000000000000002, 0.0, 0.0, 4.999999999999994]

In [15]:
y

array([  2.00000000e+00,   2.00000000e+00,   1.77635684e-16])

In [21]:
A.shape[1]

3